In [1]:
import sys
sys.path.append('../..')

In [2]:
import torch

from src.tool import config
from src import dataset, dataloader, model, loss, metric, optimizer
from src.tool.registry import DATASET_REGISTRY, DATALOADER_REGISTRY, MODEL_REGISTRY, LOSS_REGISTRY, METRIC_REGISTRY, OPTIMIZER_REGISTRY

# torch settings
torch.manual_seed(0)

if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

device

'mps'

In [3]:
# load options
path = '../../experiment/20210615/'
opt = config.load_config(path)
opt

CfgNode({'dataset': CfgNode({'name': 'Pedar_Dataset_static2dynamic', 'train_ratio': 0.8, 'args': CfgNode({'pedar_static': 'data/processed/pedar_static.pkl', 'pedar_dynamic': 'data/processed/pedar_dynamic.pkl', 'sense_range': 600})}), 'dataloader': CfgNode({'name': 'DataLoader', 'args': CfgNode({'batch_size': 32, 'shuffle': True})}), 'model': CfgNode({'name': 'MLP', 'args': CfgNode({'input_size': 198, 'hidden_size': 256, 'output_size': 198})}), 'loss': CfgNode({'name': 'MSELoss', 'args': CfgNode({})}), 'metric': CfgNode({'MAE': CfgNode({'name': 'L1Loss', 'args': CfgNode({})})}), 'optimizer': CfgNode({'name': 'SGD', 'epoch': 100, 'args': CfgNode({'lr': 0.1})})})

In [4]:
opt.dataset.args.pedar_static = '../../data/processed/pedar_static.pkl'
opt.dataset.args.pedar_dynamic = '../../data/processed/pedar_dynamic.pkl'
opt

CfgNode({'dataset': CfgNode({'name': 'Pedar_Dataset_static2dynamic', 'train_ratio': 0.8, 'args': CfgNode({'pedar_static': '../../data/processed/pedar_static.pkl', 'pedar_dynamic': '../../data/processed/pedar_dynamic.pkl', 'sense_range': 600})}), 'dataloader': CfgNode({'name': 'DataLoader', 'args': CfgNode({'batch_size': 32, 'shuffle': True})}), 'model': CfgNode({'name': 'MLP', 'args': CfgNode({'input_size': 198, 'hidden_size': 256, 'output_size': 198})}), 'loss': CfgNode({'name': 'MSELoss', 'args': CfgNode({})}), 'metric': CfgNode({'MAE': CfgNode({'name': 'L1Loss', 'args': CfgNode({})})}), 'optimizer': CfgNode({'name': 'SGD', 'epoch': 100, 'args': CfgNode({'lr': 0.1})})})

## Data loading

In [5]:
from torch.utils.data import random_split

full_dataset = DATASET_REGISTRY[opt.dataset.name](**opt.dataset.args)
train_dataset, test_dataset = random_split(full_dataset, [opt.dataset.train_ratio, 1 - opt.dataset.train_ratio])

train_dataloader = DATALOADER_REGISTRY[opt.dataloader.name](train_dataset, **opt.dataloader.args)
test_dataloader = DATALOADER_REGISTRY[opt.dataloader.name](test_dataset, **opt.dataloader.args)

## Launch training

In [6]:
from src.tool import train

model = MODEL_REGISTRY[opt.model.name](**opt.model.args).to(device)
loss = LOSS_REGISTRY[opt.loss.name](**opt.loss.args)
optimizer = OPTIMIZER_REGISTRY[opt.optimizer.name](**opt.optimizer.args, params=model.parameters())

save_interval = 10
logs = train.train_loop(path, model, train_dataloader, test_dataloader, optimizer, loss, opt.optimizer.epoch, device, save_interval)

  0%|          | 0/100 [00:00<?, ?it/s]

model saved to ../../experiment/20210615/model.pth
model and optimizer saved ../../experiment/20210615/optimizer.pth
epoch: 1 | train_loss: 0.1661 | test_loss: 0.1611
epoch: 2 | train_loss: 0.1554 | test_loss: 0.1506
epoch: 3 | train_loss: 0.1454 | test_loss: 0.1409
epoch: 4 | train_loss: 0.1356 | test_loss: 0.1317
epoch: 5 | train_loss: 0.1268 | test_loss: 0.1231
epoch: 6 | train_loss: 0.1184 | test_loss: 0.1150
epoch: 7 | train_loss: 0.1108 | test_loss: 0.1075
epoch: 8 | train_loss: 0.1033 | test_loss: 0.1003
epoch: 9 | train_loss: 0.0966 | test_loss: 0.0936
epoch: 10 | train_loss: 0.0903 | test_loss: 0.0875
model saved to ../../experiment/20210615/model.pth
model and optimizer saved ../../experiment/20210615/optimizer.pth
epoch: 11 | train_loss: 0.0841 | test_loss: 0.0817
epoch: 12 | train_loss: 0.0784 | test_loss: 0.0764
epoch: 13 | train_loss: 0.0733 | test_loss: 0.0714
epoch: 14 | train_loss: 0.0685 | test_loss: 0.0666
epoch: 15 | train_loss: 0.0640 | test_loss: 0.0622
epoch: 16 